In [1]:
import sqlite3
import pickle
import pandas as pd

In [ ]:
shutuba_table = pd.read_pickle("syutuba_table_test.pickle")
# shutuba_table.drop(columns=['斤量','jockey_id', 'trainer_id', 'トレーナー名', '体重'\
#                             , '体重変化', 'n_horses', 'course_len', 'weather' \
#                             , 'race_type', 'ground_state', 'around'], inplace=True)

In [ ]:
shutuba_table = shutuba_table.astype(str) # 全体を文字列に
shutuba_table.reset_index(inplace=True)
shutuba_table = shutuba_table.rename(columns={'index': 'race_id'}) # インデックスを列に

In [ ]:
conn = sqlite3.connect('races.sqlite')
cursor = conn.cursor()

**********

predictsテーブル

In [ ]:
pred_table = shutuba_table[['race_id', '馬番', 'score1' '...', 'score8']].copy()

In [ ]:
pred_table.rename(columns={'馬番' : 'runners_number', 
                           'score1' : 'model1',
                            'score2' : 'model2',
                            'score3' : 'model3',
                            'score4' : 'model4',}, 
                  inplace=True)

In [ ]:
pred_table.to_sql('predicts', conn, if_exists='append', index=None)

*************

In [ ]:
conn.close()